# Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Constants

In [ ]:
# minimum probability to filter weak detections 
MIN_CONF = 0.3
# threshold when applying non-maxima suppression
NMS_THRESH = 0.3
# minimum safe distance (in pixels) 
MIN_DISTANCE = 50

# Destect People

In [ ]:
import numpy as np
import cv2

def detect_people(frame, net, ln, personIdx=0):
	print("Detect People Starts")
	# frame dimensions   
	(height, width) = frame.shape[:2]
	results = []

	# construct a blob from the input frame, pre-process image using blobFromImage
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
 	# perform a forward pass of the YOLO object detector, giving us our bounding boxes
	# and associated probabilities
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	boxes = []
	centroids = []
	confidences = []

	for output in layerOutputs:
		for detection in output:
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# ensuring that the object detected was a person that the minimum
			# confidence is met
			if classID == personIdx and confidence > MIN_CONF:
				# scale the bounding box coordinates back relative to
				# the size of the image
				box = detection[0:4] * np.array([width, height, width, height])
				# YOLO returns 
				(centerX, centerY, width, height) = box.astype("int")

				# calculate top left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	# Performs non maximum suppression given boxes and corresponding scores,
	# to suppress weak, overlapping bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

	# ensure we detected something using non maximum suppression
	if len(idxs) > 0:
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			res = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(res)
	print("Detect People Ends")
	return results

# YOLOv3

In [ ]:
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

path = '/content/drive/My Drive/CV_Final_Proj/yolo-coco'
gpu= False

# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([path, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")
input_video = '/content/drive/My Drive/CV_Final_Proj/walking.mp4'
output_video = '/content/drive/My Drive/CV_Final_Proj/walking_output2.avi'
display = False

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([path, "yolov3.weights"])
configPath = os.path.sep.join([path, "yolov3.cfg"])

# load YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# check if we are going to use GPU
if gpu:
	# set CUDA as the preferable backend and target
	print("[INFO] setting preferable backend and target to CUDA...")
	net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
	net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream and pointer to output video file
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(input_video if input_video else 0)
writer = None

# loop over the frames from the video stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break

	# resize the frame  
	frame = imutils.resize(frame, width=700)
 	# detect only people in frame
	results = detect_people(frame, net, ln,
		personIdx=LABELS.index("person"))

	# Aset of indexes that violate the minimum social distance
	violate = set()

	# ensure there are at least two people detections
	if len(results) >= 2:
		# extract all centroids from the results 
		centroids = np.array([r[2] for r in results])
		# Compute the Euclidean distances between all pairs of the centroids
		D = dist.cdist(centroids, centroids, metric="euclidean")

		# loop over the upper triangular of the distance matrix
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				# distance between any two centroid pairs is less than the configured
				#  number of pixels
				if D[i, j] < MIN_DISTANCE:
					# update violation set with the indexes of the centroid pairs
					violate.add(i)
					violate.add(j)

	for (i, (prob, bbox, centroid)) in enumerate(results):
		# bounding box coordinates
		(startX, startY, endX, endY) = bbox
		# centroid coordinates
		(cX, cY) = centroid
		# color of the annotation
		color = (0, 255, 0)

		# update the color if theres a violation
		if i in violate:
			color = (0, 0, 255)

		# draw a bounding box around the person
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.circle(frame, (cX, cY), 5, color, 1)

	# draw the total number of social distancing violations 
	text = "Total Number of Social Distancing Violations: {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

	if display> 0:
		# show the output frame
		cv2.imshow("Frame", frame)
		key = cv2.waitKey(1) & 0xFF

		# # if the `q` key was pressed, break from the loop
		# if key == ord("q"):
		# 	break

	# if an output video file path has been supplied and the video
	# writer has not been initialized
	if output_video != "" and writer is None:
		print("Initialize Video writer")
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    
		writer = cv2.VideoWriter(output_video, fourcc, 25,
			(frame.shape[1], frame.shape[0]), True)

	# if the video writer is not None, write the frame to the output
	# video file
	if writer is not None:
		print("Write frame to output")
		writer.write(frame)

[INFO] loading YOLO from disk...
[INFO] accessing video stream...
Detect People Starts
Detect People Ends
Initialize Video writer
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Write frame to output
Detect People Starts
Detect People Ends
Wr

# For GPU Usage

In [ ]:
!pip install --user  opencv-python --upgrade

     |████████████████████████████████| 43.7MB 69kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
